# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('output_data/weather_data.csv')


In [3]:
weather_data.head()

,city,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed,Country
0,albany,42.6001,-73.9662,63.36,94,100,1.01,US
1,cape town,-33.9258,18.4232,55.60,57,89,5.99,ZA
2,hobart,-42.8794,147.3294,55.42,68,75,5.99,AU
3,atuona,-9.8000,-139.0333,77.97,78,32,21.18,PF
4,guerrero negro,27.9769,-114.0611,71.82,58,26,3.76,MX


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = weather_data[['Lat','Long']]
humidity = weather_data['Humidity']
max_humid = humidity.max()
max_humid

100

In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)



In [6]:
# Create a humidity heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Ideal weather conditions

ideal_tmpHigh = 85
ideal_tmpLow = 75
ideal_wind = 10
ideal_cloud = 10
ideal_humid = 40

In [9]:
# locate cities that fit ideal weather conditions

ideal_weatherDF = weather_data[(weather_data['Temp'] >= ideal_tmpLow) & 
                               (weather_data['Temp'] <= ideal_tmpHigh) &
#                                (weather_data['Wind Speed'] <= ideal_wind) &
                               (weather_data['Cloudiness'] <= ideal_cloud) &
                               (weather_data['Humidity'] <= ideal_humid)]

ideal_weatherDF


,city,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed,Country
25,buraydah,26.3260,43.9750,84.00,15,0,6.06,SA
66,riyadh,24.6877,46.7219,82.54,17,0,6.71,SA
75,dori,14.0354,-0.0345,82.72,36,5,7.81,BF
146,aswan,24.0934,32.9070,81.70,21,0,11.50,EG
153,ambilobe,-13.2000,49.0500,82.94,34,0,14.29,MG
184,byron bay,-28.6500,153.6167,78.28,36,0,6.76,AU
208,araouane,18.9048,-3.5265,82.54,10,1,9.78,ML
312,joshimath,30.5667,79.5667,76.44,17,0,6.62,IN
401,namwala,-15.7504,26.4384,82.71,25,1,6.35,ZM
426,kutum,14.2000,24.6667,80.76,30,5,11.16,SD


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
# make hotel dataframe
hotel_df = ideal_weatherDF.copy()
hotel_df['Hotel Name'] = ''

hotel_df

,city,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
25,buraydah,26.3260,43.9750,84.00,15,0,6.06,SA,
66,riyadh,24.6877,46.7219,82.54,17,0,6.71,SA,
75,dori,14.0354,-0.0345,82.72,36,5,7.81,BF,
146,aswan,24.0934,32.9070,81.70,21,0,11.50,EG,
153,ambilobe,-13.2000,49.0500,82.94,34,0,14.29,MG,
184,byron bay,-28.6500,153.6167,78.28,36,0,6.76,AU,
208,araouane,18.9048,-3.5265,82.54,10,1,9.78,ML,
312,joshimath,30.5667,79.5667,76.44,17,0,6.62,IN,
401,namwala,-15.7504,26.4384,82.71,25,1,6.35,ZM,
426,kutum,14.2000,24.6667,80.76,30,5,11.16,SD,


In [11]:
# set search parameters for hotel search 
search_radius = 10000
search_type = 'lodging'
hotel_names = []

In [12]:
# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [13]:
# iterate over each row in hotel_df
for index,row in hotel_df.iterrows():
    try:
        
        # pull lat, long data
        lat = row['Lat']
        lng = row['Long']
        
        coords = f'{lat},{lng}'
        
        # set parameters for google places api
        params = {
        "location": coords,
        "keyword": 'hotel',
        "radius": search_radius,
        "type": search_type,
        "key": g_key
        }

        response = requests.get(base_url, params=params)
        places_data = response.json()
        
        hotel_name = (places_data['results'][0]['name'])
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        
        print(hotel_name)
        
    except:
        hotel_df.loc[index, 'Hotel Name'] = 'None'
        City = row['city']
        print(f'Error - did not find hotel in {City}')
    

Best Western Plus Buraidah
Radisson Blu Hotel, Riyadh
Dintal Hotel
Sofitel Legend Old Cataract Aswan
Hotel National
Byron Bay Hotel and Apartments - Boutique Accommodation
Error - did not find hotel in araouane
The Tattva - A Boutique Resort
Karibu Guest House
Error - did not find hotel in kutum
Error - did not find hotel in tessalit
Gladstone Reef Hotel
Hôtel Temet


In [14]:
# drop rows where hotels could not be found
hotel_df = hotel_df[hotel_df['Hotel Name'] != 'None']

hotel_df

,city,Lat,Long,Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
25,buraydah,26.3260,43.9750,84.00,15,0,6.06,SA,Best Western Plus Buraidah
66,riyadh,24.6877,46.7219,82.54,17,0,6.71,SA,"Radisson Blu Hotel, Riyadh"
75,dori,14.0354,-0.0345,82.72,36,5,7.81,BF,Dintal Hotel
146,aswan,24.0934,32.9070,81.70,21,0,11.50,EG,Sofitel Legend Old Cataract Aswan
153,ambilobe,-13.2000,49.0500,82.94,34,0,14.29,MG,Hotel National
184,byron bay,-28.6500,153.6167,78.28,36,0,6.76,AU,Byron Bay Hotel and Apartments - Boutique Acco...
312,joshimath,30.5667,79.5667,76.44,17,0,6.62,IN,The Tattva - A Boutique Resort
401,namwala,-15.7504,26.4384,82.71,25,1,6.35,ZM,Karibu Guest House
512,gladstone,-23.8500,151.2500,82.33,26,0,13.80,AU,Gladstone Reef Hotel
523,arlit,18.7369,7.3853,81.81,14,5,9.66,NE,Hôtel Temet


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [17]:
# get list of coordinates
locations = hotel_df[["Lat", "Long"]]

# put into list of tuples
locations = locations.to_records(index=False)
locations = list(locations)
locations


[(26.326, 43.975),
 (24.6877, 46.7219),
 (14.0354, -0.0345),
 (24.0934, 32.907),
 (-13.2, 49.05),
 (-28.65, 153.6167),
 (30.5667, 79.5667),
 (-15.7504, 26.4384),
 (-23.85, 151.25),
 (18.7369, 7.3853)]

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, hotel_info)

# fig.add_layer(markers)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))